Intermediate Operations:

```java
filter(Predicate<T>)
map(Function<T>)
flatMap(Function<T>)
sorted(Comparator<T>)
peek(Consumer<T>)
distinct()
limit(long n)
skip(long n)
```

Terminal Operations:

```java
forEach
forEachOrdered
toArray
reduce
collect
min
max
count
anyMatch
allMatch
noneMatch
findFirst    
findAny
```

# Java 8 Streams
----

First of all, Java 8 Streams should not be confused with Java I/O streams (ex: `FileInputStream` etc); these have very little to do with each other. This functionality – `java.util.stream` – adds support **functional-style** operations on streams of elements, such as `map`, `reduce` on collections. https://stackify.com/streams-guide-java-8/

Suppose we want to iterate over a list of integers and find out sum of all the integers greater than 10. Prior to Java 8, the approach to do it would be:

```java
private static int sumIterator(List<Integer> list) {
	Iterator<Integer> it = list.iterator();
	int sum = 0;
	while (it.hasNext()) {
		int num = it.next();
		if (num > 10) {
			sum += num;
		}
	}
	return sum;
}
```

- We just want to know the sum of integers but we would also have to provide how the iteration will take place, this is also called **external iteration** because client program is handling the algorithm to iterate over the list.
- The program is sequential in nature, there is no way we can do this in parallel easily.
- There is a lot of code to do even a simple task.

To overcome all the above shortcomings, Java 8 Stream API was introduced. We can use Java Stream API to implement **internal iteration**, that is better because java framework is in control of the iteration. **Internal iteration** provides several features such as sequential and parallel execution, filtering based on the given criteria, mapping etc. 

Most of the Java 8 Stream API method arguments are functional interfaces, so lambda expressions work very well with them. 

```java
private static int sumStream(List<Integer> list) {
	return list.stream().filter(i -> i > 10).mapToInt(i -> i).sum();
}
```

Since we can use primitive data types such as `int`, `long` in the collections using **auto-boxing** and these operations could take a lot of time, there are specific classes for primitive types - `IntStream`, `LongStream` and `DoubleStream`.


# Intermediate & Terminal Operations
---

Java Stream API operations that returns a new Stream are called intermediate operations. Most of the times, these operations are lazy in nature, so they start producing new stream elements and send it to the next operation. Intermediate operations are never the final result producing operations. Commonly used intermediate operations are `filter` and `map`. Java 8 Stream API operations that returns a result or produce a side effect. Once the terminal method is called on a stream, **it consumes the stream** and after that we can’t use stream. Commonly used terminal methods are `forEach`, `toArray`, `reduce`, `max`, `anyMatch` etc. You can identify terminal methods from the return type, they will never return a Stream. 

上面那句Terminal Operations会consume Stream, 然后再看: 

Java Streams are consumable, so there is no way to create a reference to stream for future usage. Since the data is on-demand, **it’s not possible to reuse** the same stream multiple times. 

再看collection和stream的对比:

A **collection** is an in-memory data structure to hold values and before we start using collection, all the values should have been populated. Whereas a java **Stream** doesn’t store data, it operates on the source data structure (collection and array) and **produce pipelined data** that we can use and perform specific operations(`reduce()`, `toArray()`, `max()`).

参考: https://www.digitalocean.com/community/tutorials/java-8-stream#stream-intermediate-terminal-operations

### 来看个例子

这个程序的目的是分组计数, 就是想统计一下多少岁的各有多少人, 最后打印类似`{12: 3, 15: 1}`的数据. 

```java
class Person {
    public String name;
    public int age;

    public Person(int age, String name) {
        this.age = age;
        this.name = name;
    }
}

public class Main {
    public static void main(String[] args) {
        ArrayList<Person> arr = new ArrayList<>();
        arr.add(new Person(12, "jack"));
        arr.add(new Person(15, "mack"));
        arr.add(new Person(12, "john"));
        arr.add(new Person(12, "yhon"));

        HashMap<Integer, Integer> json = new HashMap<>();
        arr.stream().peek((ele) -> {
            if (!json.containsKey(ele.age)) {
                json.put(ele.age, 1);
            } else {
                json.put(ele.age, json.get(ele.age)+1);
            }
        });
        System.out.println(json);
    }
}
```

上面这个程序, 没有报错, 看着很正常, 可是, 最后却打印了`{}`, 也就是说HashMap是空的! 这是怎么回事, 难道if判断条件有问题吗? 在if里打印点东西试试: 

```java
arr.stream().peek((ele) -> {
            if (!json.containsKey(ele.age)) {
                System.out.println("put");
                json.put(ele.age, 1);
            } else {
                System.out.println("hello");
                json.put(ele.age, json.get(ele.age)+1);
            }
        });
System.out.println(json);
```

你觉得这次至少打印点东西了吧, 其实, 什么都没打印, 还是只打印了`{}`!

这是咋回事? 看看上面说的,  

Java Stream is a data structure that is **computed on-demand**. All intermediate operations are lazy, and, as a result, **no operations will have any effect until the pipeline starts to work**. peek()是一个intermediate operation. 

所以原因就知道了: 直到我们apply terminal operation to a Stream的时候, 之前应用到pipeline的操作(peek, map, etc)才会开始工作: it's an intermediate operation and we didn't apply a terminal operation to the pipeline. 

所以解决办法很简单, 就是对peek产生的stream应用一个terminal operation, 比如forEach. 

```java
ArrayList<Person> arr = new ArrayList<>();
        arr.add(new Person(12, "jack"));
        arr.add(new Person(15, "mack"));
        arr.add(new Person(12, "john"));
        arr.add(new Person(12, "yhon"));

        HashMap<Integer, Integer> json = new HashMap<>();
        arr.stream().peek((ele) -> {
            if (!json.containsKey(ele.age)) {
                json.put(ele.age, 1);
            } else {
                json.put(ele.age, json.get(ele.age)+1);
            }
        }).forEach(System.out::print);
System.out.println();
System.out.println(json);

打印:
Person@372f7a8dPerson@2f92e0f4Person@28a418fcPerson@5305068a
{12=3, 15=1}
```

参考: https://www.baeldung.com/java-streams-peek-api


# Java Stream Examples
----

## Creating Java Streams

There are several ways through which we can create a java stream from array and collections.

```java
Stream<Integer> stream = Stream.of(1,2,3,4);

Stream<Integer> stream = Stream.of(new Integer[]{1,2,3,4}); 
//works fine

Stream<Integer> stream1 = Stream.of(new int[]{1,2,3,4}); 
//Compile time error, Type mismatch: cannot convert from Stream<int[]> to Stream<Integer> 
//Note that it doesn’t support autoboxing, so we can’t pass primitive type array.
```
We can use Collection `stream()` to create sequential stream and `parallelStream()` to create parallel stream.

```java
List<Integer> myList = new ArrayList<>();
for(int i=0; i<100; i++) myList.add(i);

//sequential stream
Stream<Integer> sequentialStream = myList.stream();

//parallel stream
Stream<Integer> parallelStream = myList.parallelStream();
```

## Java Stream Intermediate Operations

### 1) filter() 

> Using `filter()` to Select Values from an Array. 

`filter()`返回的是一个stream, 参数是一个函数. 在js里filter()是这样的: 参数是匿名函数, 该匿名函数体内应该是一个逻辑判断, 当这个条件为真的时候, 就会把当前元素复制到新的数组(比如你可以在函数体写个`true`, 那每个元素都会被复制到新的集合里), 这种肯定是浅拷贝. **元素个数有可能会变**.

```java
List<Integer> myList = new ArrayList<>();
for(int i=0; i<100; i++) myList.add(i);
Stream<Integer> sequentialStream = myList.stream();

Stream<Integer> highNums = sequentialStream.filter(p -> p > 90); 
System.out.print("High Nums greater than 90=");
highNums.forEach(p -> System.out.print(p+" "));
//prints "High Nums greater than 90=91 92 93 94 95 96 97 98 99 "
```


### 2) map()

map()相对filter()就很好理解, 同样map()的参数也是函数, 然后同样返回一个新的stream, 函数体里的语句会对集合里的每个元素应用一次, 所以这个更好用一些其实, 但是无论怎样, map执行之后, **集合里的元素个数不会多也不会少**. 

```java
Stream<String> names = Stream.of("aBc", "d", "ef");
System.out.println(names.map(s -> {
		return s.toUpperCase();
	}).collect(Collectors.toList()));
//prints [ABC, D, EF]
```

看到这你可能会发现map(), filter()返回的都是stream, 那怎么把stream转成collection呢?

We can use java Stream `collect()` method to get List, Map or Set from stream.

We can use stream `toArray()` method to create an array from the stream.

具体操作可以谷歌查一下. 

### 3) reduce()

> Using `reduce()` to Turn an Array into a Single Value. reduce方法按顺序对数组每个元素执行某个函数(accumulator)，这个函数接收上一次执行结果作为参数. 

- identity – the identity value for the accumulating function 
- accumulator – a stateless function for combining two values 

`accumulator`应该接受两个参数: 第一个参数的初始值其实就是`identity`, 然后每次该函数执行的结果会传递给该函数的第一个参数来执行下一次, 第二个参数就是集合的当前元素, 然后accumulator最后返回结果.

```java
T reduce(T identity, BinaryOperator<T> accumulator);
```

### 4) sorted()

```java
Stream<String> names2 = Stream.of("aBc", "d", "ef", "123456");
List<String> reverseSorted = names2.sorted(Comparator.reverseOrder()).collect(Collectors.toList());
System.out.println(reverseSorted); // [ef, d, aBc, 123456]
```

### 5) forEach() 

This can be used for iterating over the stream.

```java
Stream<Integer> numbers2 = Stream.of(1,2,3,4,5);
numbers2.forEach(i -> System.out.print(i+",")); //1,2,3,4,5,
```


# When to Use a Parallel Stream
---

It's also very easy to create streams that execute in parallel and make use of multiple processor cores. We might think that it's always faster to divide the work on more cores. But that is often not the case.

## Sequential Streams
By default, any stream operation in Java is processed sequentially, unless explicitly specified as parallel. Sequential streams use a single thread to process the pipeline:

```java
List<Integer> listOfNumbers = Arrays.asList(1, 2, 3);
listOfNumbers.stream().forEach(number ->
    System.out.println(number + " " + Thread.currentThread().getName())
);

1 main
2 main
3 main
```

## Parallel Streams
We can achieve this by adding the parallel method to a sequential stream or by creating a stream using the parallelStream method of a collection:

```java
List<Integer> listOfNumbers = Arrays.asList(1, 2, 3, 4);
listOfNumbers.parallelStream().forEach(number ->
    System.out.println(number + " " + Thread.currentThread().getName())
);

4 ForkJoinPool.commonPool-worker-3
2 ForkJoinPool.commonPool-worker-5
1 ForkJoinPool.commonPool-worker-7
3 main
```

Parallel streams enable us to execute code in parallel on separate cores. The final result is the combination of each individual outcome. However, the order of execution is out of our control. It may change every time we run the program.

后面的因为涉及到并发框架, 现在还没了解, 想了解更多可以看下面这篇文章

Original Blog: 
https://www.baeldung.com/java-when-to-use-parallel-stream/